In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
vistas = pd.read_csv('../data/adri_data/entrenamiento_vistas2.csv');
vistas.head()

,idaviso,idpostulante,year,month,day,dayofweek,hour,valor_educacion,educacion_int,sexo,edad,cantidad_postulaciones_x,cantidad_vistas_x,cantidad_interacciones,esta_online,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,cantidad_postulaciones_y,cantidad_vistas_y
0,1112400775,kPjr2K0,2018,4,6,4,5,9,16,1,20,21,54,75,-1,1,5,182,4999,4281
1,1112400775,kPjr2K0,2018,4,6,4,5,9,16,1,20,21,54,75,-1,1,5,182,4999,4281
2,1112400775,kPjr2K0,2018,4,6,4,5,9,16,1,20,21,54,75,-1,1,5,182,4999,4281
3,1112400775,rmRpqEA,2018,4,4,2,2,12,20,1,33,174,566,740,-1,1,5,182,4999,4281
4,1112400775,EzEovD2,2018,4,6,4,5,4,12,1,22,6,8,14,-1,1,5,182,4999,4281


In [3]:
postulaciones = pd.read_csv('../data/adri_data/entrenamiento_postulaciones2.csv');
postulaciones.head()

,idaviso,esta_online,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,cantidad_postulaciones_x,cantidad_vistas_x,idpostulante,month,day,dayofweek,hour,valor_educacion,educacion_int,sexo,edad,cantidad_postulaciones_y,cantidad_vistas_y,cantidad_interacciones
0,1111556097,-1,1,5,50,227,239,pzeVW63,3,13,1,16,17,11,1,44,30,25,55
1,1112370711,1,1,5,143,386,143,pzeVW63,3,8,3,14,17,11,1,44,30,25,55
2,1112326655,1,1,5,29,362,43,pzeVW63,2,9,4,14,17,11,1,44,30,25,55
3,1112350182,1,1,5,119,541,175,pzeVW63,2,20,1,15,17,11,1,44,30,25,55
4,1112383837,1,1,5,115,223,93,pzeVW63,3,7,2,14,17,11,1,44,30,25,55


In [4]:
postulaciones.columns

Index(['idaviso', 'esta_online', 'idtipo_de_trabajo_int',
       'idnivel_laboral_int', 'idnombre_area_int', 'cantidad_postulaciones_x',
       'cantidad_vistas_x', 'idpostulante', 'month', 'day', 'dayofweek',
       'hour', 'valor_educacion', 'educacion_int', 'sexo', 'edad',
       'cantidad_postulaciones_y', 'cantidad_vistas_y',
       'cantidad_interacciones'],
      dtype='object')

In [5]:
vistas = vistas[postulaciones.columns]

In [6]:
postulaciones = postulaciones[postulaciones.columns]

In [7]:
vistas['se_postulo'] = 0
postulaciones['se_postulo'] = 1

In [8]:
vistas.shape

(10471713, 20)

In [9]:
postulaciones.shape

(6188634, 20)

In [10]:
vistas_y_postulaciones = pd.concat([postulaciones, vistas], axis=0)
vistas_y_postulaciones.shape

(16660347, 20)

In [11]:
vistas_y_postulaciones.head()

,idaviso,esta_online,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,cantidad_postulaciones_x,cantidad_vistas_x,idpostulante,month,day,dayofweek,hour,valor_educacion,educacion_int,sexo,edad,cantidad_postulaciones_y,cantidad_vistas_y,cantidad_interacciones,se_postulo
0,1111556097,-1,1,5,50,227,239,pzeVW63,3,13,1,16,17,11,1,44,30,25,55,1
1,1112370711,1,1,5,143,386,143,pzeVW63,3,8,3,14,17,11,1,44,30,25,55,1
2,1112326655,1,1,5,29,362,43,pzeVW63,2,9,4,14,17,11,1,44,30,25,55,1
3,1112350182,1,1,5,119,541,175,pzeVW63,2,20,1,15,17,11,1,44,30,25,55,1
4,1112383837,1,1,5,115,223,93,pzeVW63,3,7,2,14,17,11,1,44,30,25,55,1


# Agrego la tercer clase

In [12]:
top100 = pd.read_csv("../test_final_100k.csv", encoding='utf-8')
top100.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [13]:
top100_full = pd.merge(top100, vistas_y_postulaciones, how='left', on=['idaviso', 'idpostulante'])
top100_full.head()

,id,idaviso,idpostulante,esta_online,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,cantidad_postulaciones_x,cantidad_vistas_x,month,...,dayofweek,hour,valor_educacion,educacion_int,sexo,edad,cantidad_postulaciones_y,cantidad_vistas_y,cantidad_interacciones,se_postulo
0,0,739260,6M9ZQR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,739260,6v1xdL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,739260,ezRKm9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,758580,1Q35ej,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,758580,EAN4J6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
excluded = top100_full[top100_full['esta_online'].isnull()]
excluded.shape

(60066, 21)

In [15]:
excluded = excluded[['id', 'idaviso', 'idpostulante']]
excluded.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [16]:
avisos_detalles = pd.read_csv('../data/adri_data/avisos_detalles.csv', encoding='utf-8')
avisos_detalles.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,iddenominacion_empresa_int
0,1111556097,-1,1120986480640,8230659354068575249,8159438729029635585,7,0,0,1,5,50,3680
1,1112448447,1,576460769483300864,6228765605496356872,6157568899156279424,7,0,0,1,5,182,3680
2,1112394600,-1,1152921526081683456,7063905988445249617,6991840908279038473,1,0,920,1,5,34,3680
3,1111266888,1,18014398511579136,8106761422736344128,8107605375203478536,7,0,0,1,2,142,3680
4,1112426827,1,1125921381679104,8380078581765668944,8379514742535897097,7,0,0,1,5,114,3680


In [17]:
excluded = pd.merge(excluded, avisos_detalles, how='inner', on='idaviso')
excluded.shape

(60066, 14)

In [18]:
postulantes = pd.read_csv('../data/adri_data/postulantes2.csv', encoding='utf-8')
postulantes.shape

(504407, 9)

In [19]:
excluded = pd.merge(excluded, postulantes, how='inner', on='idpostulante')
excluded.shape

(60066, 22)

In [20]:
excluded['se_postulo'] = -1

In [22]:
excluded['month'] = 0
excluded['day'] = 0
excluded['dayofweek'] = 0
excluded['hour'] = 0

excluded['cantidad_postulaciones_x'] = 0
excluded['cantidad_vistas_x'] = 0
excluded['cantidad_postulaciones_y'] = 0
excluded['cantidad_vistas_y'] = 0
excluded['cantidad_interacciones'] = 0

excluded = excluded[vistas_y_postulaciones.columns]
excluded.head()

,idaviso,esta_online,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,cantidad_postulaciones_x,cantidad_vistas_x,idpostulante,month,day,dayofweek,hour,valor_educacion,educacion_int,sexo,edad,cantidad_postulaciones_y,cantidad_vistas_y,cantidad_interacciones,se_postulo
0,739260,-1,1,2,29,0,0,6M9ZQR,0,0,0,0,21,2.0,1,42,0,0,0,-1
1,739260,-1,1,2,29,0,0,6v1xdL,0,0,0,0,11,17.0,2,30,0,0,0,-1
2,739260,-1,1,2,29,0,0,ezRKm9,0,0,0,0,12,20.0,1,36,0,0,0,-1
3,758580,-1,1,4,168,0,0,1Q35ej,0,0,0,0,17,11.0,2,68,0,0,0,-1
4,758580,-1,1,4,168,0,0,EAN4J6,0,0,0,0,10,19.0,1,32,0,0,0,-1


In [23]:
excluded = excluded[vistas_y_postulaciones.columns]

In [22]:
# timestamps = pd.DataFrame(vistas_y_postulaciones.groupby('idaviso')['timestamp'].mean()).reset_index()
# timestamps.head()

In [23]:
# excluded = pd.merge(excluded, timestamps, on='idaviso', how='left')
# excluded.head()

In [24]:
# excluded = excluded[vistas_y_postulaciones.columns]
# excluded.head()

In [25]:
# timestampmean = vistas_y_postulaciones['timestamp'].mean()
# timestampmean

In [26]:
# excluded.loc[excluded['timestamp'].isnull(), 'timestamp'] = timestampmean
# excluded.head()

In [27]:
# excluded[excluded['timestamp'].isnull()].head()

In [28]:
# excluded.loc[excluded['timestamp'].isnull(), 'timestamp'] = timestampmean
# excluded.head()

# Mergeo con la tercer clase

In [24]:
vistas_y_postulaciones = pd.concat([vistas_y_postulaciones, excluded], axis=0)
vistas_y_postulaciones.head()

,idaviso,esta_online,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,cantidad_postulaciones_x,cantidad_vistas_x,idpostulante,month,day,dayofweek,hour,valor_educacion,educacion_int,sexo,edad,cantidad_postulaciones_y,cantidad_vistas_y,cantidad_interacciones,se_postulo
0,1111556097,-1,1,5,50,227,239,pzeVW63,3,13,1,16,17,11.0,1,44,30,25,55,1
1,1112370711,1,1,5,143,386,143,pzeVW63,3,8,3,14,17,11.0,1,44,30,25,55,1
2,1112326655,1,1,5,29,362,43,pzeVW63,2,9,4,14,17,11.0,1,44,30,25,55,1
3,1112350182,1,1,5,119,541,175,pzeVW63,2,20,1,15,17,11.0,1,44,30,25,55,1
4,1112383837,1,1,5,115,223,93,pzeVW63,3,7,2,14,17,11.0,1,44,30,25,55,1


In [30]:
# vistas_y_postulaciones.shape

In [31]:
# postulantes_ids = vistas_y_postulaciones[["idpostulante", "idpostulante2"]].drop_duplicates()
# postulantes_ids.to_csv('../data/adri_data/entrenamiento_id_postulantes.csv', encoding='utf-8', index=False)

In [32]:
# vistas_y_postulaciones.drop('idpostulante', axis=1, inplace=True)
# vistas_y_postulaciones.rename({'idpostulante_int': 'idpostulante'}, inplace=True, axis=1)
# vistas_y_postulaciones.head()

In [33]:
# vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas2.csv', encoding='utf-8', index=False)

# Luego de mergear

In [34]:
# vistas_y_postulaciones = pd.read_csv('../data/adri_data/entrenamiento_postulaciones_vistas2.csv', encoding='utf-8')

In [25]:
vistas_y_postulaciones.shape

(16720413, 20)

In [26]:
vistas_y_postulaciones.head()

,idaviso,esta_online,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,cantidad_postulaciones_x,cantidad_vistas_x,idpostulante,month,day,dayofweek,hour,valor_educacion,educacion_int,sexo,edad,cantidad_postulaciones_y,cantidad_vistas_y,cantidad_interacciones,se_postulo
0,1111556097,-1,1,5,50,227,239,pzeVW63,3,13,1,16,17,11.0,1,44,30,25,55,1
1,1112370711,1,1,5,143,386,143,pzeVW63,3,8,3,14,17,11.0,1,44,30,25,55,1
2,1112326655,1,1,5,29,362,43,pzeVW63,2,9,4,14,17,11.0,1,44,30,25,55,1
3,1112350182,1,1,5,119,541,175,pzeVW63,2,20,1,15,17,11.0,1,44,30,25,55,1
4,1112383837,1,1,5,115,223,93,pzeVW63,3,7,2,14,17,11.0,1,44,30,25,55,1


In [27]:
# dup = vistas_y_postulaciones.duplicated(['idaviso', 'idpostulante'])
# vistas_y_postulaciones[dup].shape

In [28]:
# vistas_y_postulaciones.drop_duplicates(['idaviso', 'idpostulante'], inplace=True)
# vistas_y_postulaciones.shape

In [29]:
vistas_y_postulaciones = vistas_y_postulaciones.sample(frac=1).reset_index(drop=True)

In [30]:
vistas_y_postulaciones.shape

(16720413, 20)

In [31]:
vistas_y_postulaciones.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16720413 entries, 0 to 16720412
Data columns (total 20 columns):
idaviso                     int64
esta_online                 int64
idtipo_de_trabajo_int       int64
idnivel_laboral_int         int64
idnombre_area_int           int64
cantidad_postulaciones_x    int64
cantidad_vistas_x           int64
idpostulante                object
month                       int64
day                         int64
dayofweek                   int64
hour                        int64
valor_educacion             int64
educacion_int               float64
sexo                        int64
edad                        int64
cantidad_postulaciones_y    int64
cantidad_vistas_y           int64
cantidad_interacciones      int64
se_postulo                  int64
dtypes: float64(1), int64(18), object(1)
memory usage: 3.4 GB


In [43]:
# vistas_y_postulaciones.drop(['idnombre_area_int_pref','idnivel_laboral_int_pref','idtipo_de_trabajo_int_pref'], inplace=True, axis=1)

In [32]:
vistas_y_postulaciones.head()

,idaviso,esta_online,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,cantidad_postulaciones_x,cantidad_vistas_x,idpostulante,month,day,dayofweek,hour,valor_educacion,educacion_int,sexo,edad,cantidad_postulaciones_y,cantidad_vistas_y,cantidad_interacciones,se_postulo
0,1112424009,1,1,4,2,75,136,96Xdv15,3,26,0,19,12,20.0,1,28,875,2769,211,0
1,1112444846,1,1,5,163,52,128,8MaMbax,4,6,4,13,12,20.0,1,28,331,1641,180,0
2,1112362031,1,1,2,152,115,170,vVDPQoE,2,27,1,18,10,19.0,1,22,1047,2024,285,0
3,1112393575,-1,1,2,38,412,1275,VNx0axE,3,13,1,7,12,20.0,2,36,24,14,38,1
4,1112467910,1,3,4,134,11,12,lDweeoj,4,16,0,21,11,17.0,2,31,101,1404,23,0


In [33]:
vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas19_full.csv', encoding='utf-8', index=False)